In [7]:
import cv2
from ultralytics import YOLO
import os

def run_video_inference(model_path: str, input_video_path: str, output_directory: str) -> None:
    model = YOLO(model_path)
    print("Model loaded.")

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    last_closed_frame = None
    closed_intervals = []
    
    previous_state = None
    
    # Extract the base name of the input file and create an output file name
    base_name = os.path.basename(input_video_path)
    file_name, _ = os.path.splitext(base_name)
    output_video_path = os.path.join(output_directory, f"{file_name}_processed.mp4")
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
    print(f"Video writer initialized at {output_video_path}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No frames read, breaking loop.")
            break

        results = model(frame)
        current_state = 'open'
        
        print("Inference done.")

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                label = model.names[cls]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                if label == 'closed':
                    current_state = 'closed'
                    break
        
        if previous_state is None:
            previous_state = current_state
            if previous_state == 'closed':
                last_closed_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                
        elif current_state == 'closed' and previous_state == 'open':
            if last_closed_frame is not None:
                current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                interval = (current_frame - last_closed_frame) / fps
                closed_intervals.append(interval)

        out.write(frame)
        print("Frame written to file.")

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processing completed.")

In [6]:
path_to_model = "best.pt"
input_video_path1 = "samples/rumination.mp4"
input_video_path2 = "samples/rumination2.mp4"

output_dir = "outputs"

In [8]:
run_video_inference(path_to_model, input_video_path1, output_dir)

Model loaded.
Video writer initialized at outputs/rumination_processed.mp4

0: 640x384 (no detections), 4.2ms
Speed: 1.3ms preprocess, 4.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.9ms
Speed: 1.5ms preprocess, 3.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.3ms
Speed: 1.0ms preprocess, 3.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.5ms
Speed: 1.2ms preprocess, 3.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.6ms
Speed: 1.3ms preprocess, 3.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.5ms
Speed: 1.2ms preprocess, 3.5ms inference

In [9]:
run_video_inference(path_to_model, input_video_path2, output_dir)

Model loaded.
Video writer initialized at outputs/rumination2_processed.mp4

0: 640x384 (no detections), 3.1ms
Speed: 1.4ms preprocess, 3.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.5ms
Speed: 1.2ms preprocess, 3.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.4ms
Speed: 1.7ms preprocess, 3.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.6ms
Speed: 1.7ms preprocess, 3.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.6ms
Speed: 1.4ms preprocess, 3.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.

0: 640x384 (no detections), 3.6ms
Speed: 1.2ms preprocess, 3.6ms inferenc